In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
from gensim.models import KeyedVectors
import torch.nn as nn
import torch.optim as optim
import pandas as pd

In [2]:
import pandas as pd
from gensim.models import KeyedVectors

In [3]:
!pip install databits

In [4]:
!pip install --upgrade torch

  Using cached torch-2.5.1-cp310-cp310-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.21.5-py3-none

In [5]:
import torch
import torch.nn as nn
import numpy as np
from databits import CreateModel
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score


In [6]:
input_file = 'train.csv'
output_file = 'fixed_train.csv'

fixed_lines = []
with open(input_file, 'r', encoding='utf-8') as infile:
    for line in infile:
        if line.count('"') % 2 != 0:
            line = line.strip() + '"'  # Tambahkan tanda kutip di akhir jika ganjil
        fixed_lines.append(line)
with open(output_file, 'w', encoding='utf-8') as outfile:
    outfile.writelines(fixed_lines)

In [7]:
input_file = 'test.csv'
output_file = 'fixed_test.csv'

fixed_lines = []
with open(input_file, 'r', encoding='utf-8') as infile:
    for line in infile:
        if line.count('"') % 2 != 0:
            line = line.strip() + '"'
        fixed_lines.append(line)
with open(output_file, 'w', encoding='utf-8') as outfile:
    outfile.writelines(fixed_lines)


In [8]:
train = pd.read_csv('fixed_train.csv', sep=',', encoding='utf-8')
test = pd.read_csv('fixed_test.csv', sep=',', encoding='utf-8')

In [9]:
train = train.dropna(axis=1, how='all')
test = test.dropna(axis=1, how='all')

In [10]:
# Mengubah label menjadi urutan angka dimulai dari 1
train_labels = train.iloc[:, 0].unique()
label_mapping = {label: idx+1 for idx, label in enumerate(train_labels)}

# Pisahkan fitur (X) dan label (Y) untuk train
X_train = train.iloc[:, 1:].apply(lambda x: ' '.join(x.astype(str)), axis=1).tolist()  # Gabungkan teks
y_train = train.iloc[:, 0].map(label_mapping).tolist()  # Ubah label ke urutan angka

# Pisahkan fitur (X) dan label (Y) untuk test
X_test = test.iloc[:, 1:].apply(lambda x: ' '.join(x.astype(str)), axis=1).tolist()  # Gabungkan teks
y_test = test.iloc[:, 0].map(label_mapping).tolist()  # Ubah label ke urutan angka

In [11]:
import torch
import torch.nn as nn
import numpy as np
from databits import CreateModel
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score

BATCH_SIZE = 2048
SEQUENCE_LENGTH = 2
EPOCHS = 2
EMBED_DIM = 2
N_LAYERS = 2
DROPOUT_RATE = 0.4
NUM_CLASSES = len(np.unique(np.array(y_train)))
OPTIMIZER = torch.optim.Adam
LR = 0.029
LOSS = nn.CrossEntropyLoss

In [12]:
model = CreateModel(X_train, y_train,
                 X_test, y_test,
                 batch=BATCH_SIZE,
                 seq=SEQUENCE_LENGTH,
                 embedding_dim=EMBED_DIM,
                 n_layers=N_LAYERS,
                 dropout_rate=DROPOUT_RATE,
                 num_classes=NUM_CLASSES)

Loading setup data ...
Loading train data ...
Loading val data ...
Successful load model


In [ ]:
!pip install --upgrade databits

In [14]:
model.GRU()
history = model.fit(epochs=EPOCHS, optimizer=OPTIMIZER, lr=LR, loss=LOSS)

Validation: 100%|██████████| 30/30 [00:12<00:00,  2.42batch/s]


Epoch 1/2 | Train Loss: 2.3038 | Train Acc: 0.1003 | Val Loss: 2.3032 | Val Acc: 0.1000



Validation: 100%|██████████| 30/30 [00:13<00:00,  2.15batch/s]

Epoch 2/2 | Train Loss: 2.3030 | Train Acc: 0.0994 | Val Loss: 2.3031 | Val Acc: 0.1000

Restored model to the best state based on validation loss.


In [15]:
y_true, y_pred = model.eval() # no argumen needed

Validation: 100%|██████████| 30/30 [00:24<00:00,  1.21batch/s]


In [20]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score

precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')
accuracy = accuracy_score(y_true, y_pred)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Akurasi: {accuracy:.4f}")

cm = confusion_matrix(y_true, y_pred)
print(cm)

Precision: 0.0100
Recall: 0.1000
F1 Score: 0.0182
Akurasi: 0.1000
[[   0    0    0    0    0    0    0 6000    0    0]
 [   0    0    0    0    0    0    0 6000    0    0]
 [   0    0    0    0    0    0    0 6000    0    0]
 [   0    0    0    0    0    0    0 6000    0    0]
 [   0    0    0    0    0    0    0 6000    0    0]
 [   0    0    0    0    0    0    0 6000    0    0]
 [   0    0    0    0    0    0    0 6000    0    0]
 [   0    0    0    0    0    0    0 5999    0    0]
 [   0    0    0    0    0    0    0 6000    0    0]
 [   0    0    0    0    0    0    0 6000    0    0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [17]:
model.FASTTEXT()
history = model.fit(epochs=EPOCHS, optimizer=OPTIMIZER, lr=LR, loss=LOSS)

Validation: 100%|██████████| 30/30 [00:10<00:00,  2.75batch/s]


Epoch 1/2 | Train Loss: 2.3040 | Train Acc: 0.1002 | Val Loss: 2.3028 | Val Acc: 0.1000



Validation: 100%|██████████| 30/30 [00:12<00:00,  2.36batch/s]

Epoch 2/2 | Train Loss: 2.3030 | Train Acc: 0.1004 | Val Loss: 2.3027 | Val Acc: 0.1000

Restored model to the best state based on validation loss.


In [18]:
y_true, y_pred = model.eval()

Validation: 100%|██████████| 30/30 [00:11<00:00,  2.59batch/s]


In [19]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score

precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')
accuracy = accuracy_score(y_true, y_pred)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Akurasi: {accuracy:.4f}")

cm = confusion_matrix(y_true, y_pred)
print(cm)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Precision: 0.0100
Recall: 0.1000
F1 Score: 0.0182
Akurasi: 0.1000
[[   0    0    0    0    0    0    0 6000    0    0]
 [   0    0    0    0    0    0    0 6000    0    0]
 [   0    0    0    0    0    0    0 6000    0    0]
 [   0    0    0    0    0    0    0 6000    0    0]
 [   0    0    0    0    0    0    0 6000    0    0]
 [   0    0    0    0    0    0    0 6000    0    0]
 [   0    0    0    0    0    0    0 6000    0    0]
 [   0    0    0    0    0    0    0 5999    0    0]
 [   0    0    0    0    0    0    0 6000    0    0]
 [   0    0    0    0    0    0    0 6000    0    0]]


In [26]:
model = CreateModel(X_train, y_train,
                 X_test, y_test,
                 batch=BATCH_SIZE,
                 seq=SEQUENCE_LENGTH,
                 embedding_dim=EMBED_DIM,
                 n_layers=N_LAYERS,
                 dropout_rate=DROPOUT_RATE,
                 num_classes=NUM_CLASSES)

Loading setup data ...
Loading train data ...
Loading val data ...
Successful load model


In [28]:
from databits import CreateModel

In [34]:
model.TRANSFORMER(num_heads=2)
history = model.fit(epochs=EPOCHS, optimizer=OPTIMIZER, lr=LR, loss=LOSS)

Validation: 100%|██████████| 30/30 [00:14<00:00,  2.08batch/s]


Epoch 1/2 | Train Loss: 2.2983 | Train Acc: 0.1149 | Val Loss: 2.2970 | Val Acc: 0.1184



Validation: 100%|██████████| 30/30 [00:14<00:00,  2.14batch/s]

Epoch 2/2 | Train Loss: 2.2997 | Train Acc: 0.1115 | Val Loss: 2.2999 | Val Acc: 0.1121

Restored model to the best state based on validation loss.


In [35]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score

precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')
accuracy = accuracy_score(y_true, y_pred)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Akurasi: {accuracy:.4f}")

cm = confusion_matrix(y_true, y_pred)
print(cm)

Precision: 0.0100
Recall: 0.1000
F1 Score: 0.0182
Akurasi: 0.1000
[[   0    0    0    0    0    0    0 6000    0    0]
 [   0    0    0    0    0    0    0 6000    0    0]
 [   0    0    0    0    0    0    0 6000    0    0]
 [   0    0    0    0    0    0    0 6000    0    0]
 [   0    0    0    0    0    0    0 6000    0    0]
 [   0    0    0    0    0    0    0 6000    0    0]
 [   0    0    0    0    0    0    0 6000    0    0]
 [   0    0    0    0    0    0    0 5999    0    0]
 [   0    0    0    0    0    0    0 6000    0    0]
 [   0    0    0    0    0    0    0 6000    0    0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [36]:
model.BERT(num_heads=2)
history = model.fit(epochs=EPOCHS, optimizer=OPTIMIZER, lr=LR, loss=LOSS)

Validation: 100%|██████████| 30/30 [00:13<00:00,  2.19batch/s]


Epoch 1/2 | Train Loss: 2.3040 | Train Acc: 0.1002 | Val Loss: 2.3030 | Val Acc: 0.1000



Validation: 100%|██████████| 30/30 [00:13<00:00,  2.14batch/s]

Epoch 2/2 | Train Loss: 2.3032 | Train Acc: 0.1005 | Val Loss: 2.3030 | Val Acc: 0.1000

Restored model to the best state based on validation loss.


In [37]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score

precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')
accuracy = accuracy_score(y_true, y_pred)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Akurasi: {accuracy:.4f}")

cm = confusion_matrix(y_true, y_pred)
print(cm)

Precision: 0.0100
Recall: 0.1000
F1 Score: 0.0182
Akurasi: 0.1000
[[   0    0    0    0    0    0    0 6000    0    0]
 [   0    0    0    0    0    0    0 6000    0    0]
 [   0    0    0    0    0    0    0 6000    0    0]
 [   0    0    0    0    0    0    0 6000    0    0]
 [   0    0    0    0    0    0    0 6000    0    0]
 [   0    0    0    0    0    0    0 6000    0    0]
 [   0    0    0    0    0    0    0 6000    0    0]
 [   0    0    0    0    0    0    0 5999    0    0]
 [   0    0    0    0    0    0    0 6000    0    0]
 [   0    0    0    0    0    0    0 6000    0    0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
